In [1]:
from google.colab import files

uploaded = files.upload()
print(uploaded.keys())

Saving train.csv to train (4).csv
Saving test.csv to test (4).csv
dict_keys(['train (4).csv', 'test (4).csv'])


In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
...,...,...,...,...,...,...,...,...,...
18519,18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert
18522,18522,NaN,Yes,1.0,0.0,Yes,5.0,2.0,Introvert


In [3]:
# automatic EDA

'''
!pip install ydata-profiling --quiet
from ydata_profiling import ProfileReport

profile = ProfileReport(train, explorative=True)
profile.to_notebook_iframe() '''

'\n!pip install ydata-profiling --quiet\nfrom ydata_profiling import ProfileReport\n\nprofile = ProfileReport(train, explorative=True)\nprofile.to_notebook_iframe() '

In [4]:
# Install required libraries if not already installed
!pip install catboost fancyimpute --quiet

import numpy as np
from fancyimpute import SoftImpute
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier

# Define numeric and categorical columns
num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
cat_cols = ['Stage_fear', 'Drained_after_socializing']

def preprocess_with_softimpute(df, encoder=None, fit_encoder=False):
    df = df.copy()

    # Encode categorical features as numbers
    if encoder is None:
        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    if fit_encoder:
        df[cat_cols] = encoder.fit_transform(df[cat_cols].astype(str))
    else:
        df[cat_cols] = encoder.transform(df[cat_cols].astype(str))

    # Combine numeric and categorical columns for imputation
    cols_to_impute = num_cols + cat_cols
    matrix = df[cols_to_impute].to_numpy()

    # Impute missing values using SoftImpute
    imputed_matrix = SoftImpute().fit_transform(matrix)

    # Put imputed values back into DataFrame
    df[cols_to_impute] = imputed_matrix

    # Convert categorical columns back to original text labels
    df[cat_cols] = encoder.inverse_transform(df[cat_cols].round().astype(int))

    return df, encoder

# Preprocess training data and fit encoder
train_processed, encoder = preprocess_with_softimpute(train, fit_encoder=True)

# Preprocess test data using the fitted encoder
test_processed, _ = preprocess_with_softimpute(test, encoder=encoder, fit_encoder=False)

# Map target variable to numeric labels
train_processed['target'] = train_processed['Personality'].map({'Introvert': 1, 'Extrovert': 0})

# Drop unnecessary columns
train_processed = train_processed.drop(['id', 'Personality'], axis=1)
test_ids = test_processed['id']
test_processed = test_processed.drop(['id', 'Personality'], axis=1, errors='ignore')

# Separate features and target for model training
X = train_processed.drop('target', axis=1)
y = train_processed['target']

# Get indices of categorical features for CatBoost
cat_features_indices = [X.columns.get_loc(col) for col in cat_cols]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SoftImpute] Max Singular Value of X_init = 1653.160175
[SoftImpute] Iter 1: observed MAE=0.170524 rank=7
[SoftImpute] Iter 2: observed MAE=0.171616 rank=7
[SoftImpute] Iter 3: observed MAE=0.172631 rank=7
[SoftImpute] Iter 4: observed MAE=0.173559 rank=7
[SoftImpute] Iter 5: observed MAE=0.174393 rank=7
[SoftImpute] Iter 6: observed MAE=0.175129 rank=7
[SoftImpute] Iter 7: observed MAE=0.175770 rank=7
[SoftImpute] Iter 8: observed MAE=0.176324 rank=7
[SoftImpute] Iter 9: observed MAE=0.176796 rank=7
[SoftImpute] Iter 10: observed MAE=0.177199 rank=7
[SoftImpute] Iter 11: observed MAE=0.177541 rank=7
[SoftImpute] Iter 12: observed MAE=0.177829 rank=7
[SoftImpute] Iter 13: observed MAE=0.178073 rank=7
[SoftImpute] Iter 14: observed MAE=0.178279 rank=7
[SoftImpute] Iter 15: observed MAE=0.178453 rank=7
[SoftImpute] Iter 16: observed MAE=0.178600 rank=7
[SoftImpute] Iter 17: observed MAE=0.178724 rank=7
[SoftImpute] Iter 18: observed MAE=0.178830 rank=7
[SoftImpute] Iter 19: observed MAE=

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SoftImpute] Max Singular Value of X_init = 956.572342
[SoftImpute] Iter 1: observed MAE=0.171063 rank=7
[SoftImpute] Iter 2: observed MAE=0.172128 rank=7
[SoftImpute] Iter 3: observed MAE=0.173115 rank=7
[SoftImpute] Iter 4: observed MAE=0.174013 rank=7
[SoftImpute] Iter 5: observed MAE=0.174817 rank=7
[SoftImpute] Iter 6: observed MAE=0.175529 rank=7
[SoftImpute] Iter 7: observed MAE=0.176153 rank=7
[SoftImpute] Iter 8: observed MAE=0.176694 rank=7
[SoftImpute] Iter 9: observed MAE=0.177160 rank=7
[SoftImpute] Iter 10: observed MAE=0.177558 rank=7
[SoftImpute] Iter 11: observed MAE=0.177897 rank=7
[SoftImpute] Iter 12: observed MAE=0.178188 rank=7
[SoftImpute] Iter 13: observed MAE=0.178436 rank=7
[SoftImpute] Iter 14: observed MAE=0.178647 rank=7
[SoftImpute] Iter 15: observed MAE=0.178826 rank=7
[SoftImpute] Iter 16: observed MAE=0.178979 rank=7
[SoftImpute] Iter 17: observed MAE=0.179109 rank=7
[SoftImpute] Iter 18: observed MAE=0.179220 rank=7
[SoftImpute] Iter 19: observed MAE=0

In [5]:
# Define CatBoost model with tuned hyperparameters
model = CatBoostClassifier(
    iterations=963,
    depth=3,
    learning_rate=0.03115,
    l2_leaf_reg=7.905,
    bootstrap_type='Bayesian',
    random_strength=0.00767,
    eval_metric='AUC',
    verbose=100,
    random_seed=42
)

# Train model on training data with categorical features specified by names
model.fit(X, y, cat_features=cat_cols)

# Alternatively, train model with categorical features specified by column indices
model.fit(X, y, cat_features=cat_features_indices)

# Predict target labels on the test dataset
preds = model.predict(test_processed)

# Prepare submission DataFrame with IDs and predicted Personality labels
submission = pd.DataFrame({
    'id': test_ids,
    'Personality': ['Introvert' if p == 1 else 'Extrovert' for p in preds]
})

# Save submission file as CSV without index
submission.to_csv('submission.csv', index=False)

# If running in Google Colab, automatically download the submission file
try:
    from google.colab import files
    files.download('submission.csv')
except:
    pass


0:	total: 62.1ms	remaining: 59.7s
100:	total: 1.71s	remaining: 14.6s
200:	total: 4.54s	remaining: 17.2s
300:	total: 7.36s	remaining: 16.2s
400:	total: 11.3s	remaining: 15.8s
500:	total: 13.1s	remaining: 12.1s
600:	total: 14.4s	remaining: 8.67s
700:	total: 15.7s	remaining: 5.86s
800:	total: 17s	remaining: 3.43s
900:	total: 18.2s	remaining: 1.25s
962:	total: 19s	remaining: 0us
0:	total: 13ms	remaining: 12.5s
100:	total: 1.26s	remaining: 10.8s
200:	total: 3.4s	remaining: 12.9s
300:	total: 4.88s	remaining: 10.7s
400:	total: 6.12s	remaining: 8.58s
500:	total: 7.37s	remaining: 6.8s
600:	total: 8.62s	remaining: 5.19s
700:	total: 9.88s	remaining: 3.69s
800:	total: 11.2s	remaining: 2.26s
900:	total: 12.4s	remaining: 855ms
962:	total: 13.2s	remaining: 0us


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>